# 3.2 DDL: CREATE TABLE

Table design principles using DDL commands, such as CREATE TABLE, DROP TABLE, ALTER TABLE, CREATE INDEX, DROP INDEX, CREATE VIEW, DROP VIEW.

Control columns values using constraints: primary keys, foreign keys, delete with CHECK, UNIQUE, NOT NULL, AUTOINCREMENT, CASCADE.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

## CREATE TABLE

In order to create a table, we need to define the table name and the columns it will contain. Each column must have a name and a data type. We can also specify constraints for each column.

```sql
CREATE TABLE [IF NOT EXISTS] [schema_name].table_name (
	column_1 data_type PRIMARY KEY,
   	column_2 data_type NOT NULL,
	column_3 data_type DEFAULT 0,
	table_constraints
) [WITHOUT ROWID];
```

In this syntax:

- `IF NOT EXISTS`: Optional. If specified, the table will only be created if it does not already exist.
- `schema_name`: Optional. The name of the schema to which the table belongs.
- `table_name`: The name of the table to be created.
- `column_1`, `column_2`, `column_3`: The names of the columns in the table.
- `data_type`: The data type of the column (e.g., INTEGER, TEXT, REAL, BLOB).
- `PRIMARY KEY`: Specifies that the column is a primary key.
- `NOT NULL`: Specifies that the column cannot contain NULL values.
- `DEFAULT`: Specifies a default value for the column if no value is provided during insertion.
- `table_constraints`: Optional. Additional constraints for the table, such as foreign keys or unique constraints.
- `WITHOUT ROWID`: Optional. Specifies that the table should not use a ROWID for each row. This is useful for tables that do not require a unique identifier for each row.

```{note}
The `WITHOUT ROWID` option is specific to SQLite and is not part of the SQL standard. It is used to create a table that does not use a ROWID for each row, which can improve performance in certain cases. By default, SQLite creates a ROWID for each row in a table, which can be used as a unique identifier. However, if you do not need a ROWID, you can use the `WITHOUT ROWID` option to create a table without it. The `rowid` column stores a 64-bit signed integer that uniquely identifies a row in a table.
```

### Example

In [2]:
%%sql
CREATE TABLE employees (
    employee_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    hire_date DATE NOT NULL,
    salary REAL CHECK (salary > 0)
);

 * sqlite:///
Done.


[]

This table `employees` has five columns:

- `employee_id`: an integer that is the primary key and will auto-increment with each new record.
- `first_name`: a text field that cannot be null.
- `last_name`: a text field that cannot be null.
- `hire_date`: a date field that cannot be null.
- `salary`: a real number that must be greater than 0, enforced by a CHECK constraint.

## Storage Classes (SQLite Data Types)

SQLite uses the following storage classes:
- NULL: The value is a NULL value.
- INTEGER: The value is a signed integer, stored in 1, 2, 3, 4, or 8 bytes.
- REAL: The value is a floating point value, stored as an 8-byte IEEE floating point number.
- TEXT: The value is a text string, stored using the database encoding (UTF-8, UTF-16BE, or UTF-16LE).
- BLOB: The value is a blob of data, stored exactly as it was input. BLOB stands for "Binary Large Object". Its size is unlimited, and it can store any kind of data, including images, audio, and video files.

SQLite determines the data type of a value based on the value itself, not the column definition. This means that you can store any type of data in any column, regardless of its declared type. However, it is a good practice to use the correct data type for each column to ensure data integrity and improve performance. For defining the data type of a column, SQLite uses the following rules:

- If a value has no enclosing quotes and decimal point or exponent, it is stored as an REAL.
- If a value has enclosing single or double quotes, it is stored as TEXT.
- If a value has no enclosing quotes and no decimal point or exponent, it is stored as INTEGER.
- If a value is NULL without enclosing quotes, it is stored as NULL.
- If a value has a prefix of X, it is stored as BLOB.

```{note}
SQLite does not have date and time classes built-in. However, it is possible to store date and time values as TEXT, REAL, or INTEGER. Check the material on date and time functions for more information.
```

You can always check the data type of a column using the `PRAGMA table_info(table_name)` command. This command returns a list of all columns in the specified table, along with their data types and constraints.

```sql
PRAGMA table_info(employees);
```

In [3]:
%%sql

PRAGMA table_info('employees');

 * sqlite:///
Done.


cid,name,type,notnull,dflt_value,pk
0,employee_id,INTEGER,0,None,1
1,first_name,TEXT,1,None,0
2,last_name,TEXT,1,None,0
3,hire_date,DATE,1,None,0
4,salary,REAL,0,None,0


You can also use `typeof()` function to check the data type of a value. This function returns the storage class of the value, which can be one of the following: NULL, INTEGER, REAL, TEXT, or BLOB.

In [4]:
%%sql
SELECT  typeof(100),
        typeof(10.0),
        typeof('100'),
        typeof(x'1000'),
        typeof(NULL);

 * sqlite:///
Done.


typeof(100),typeof(10.0),typeof('100'),typeof(x'1000'),typeof(NULL)
integer,real,text,blob,null


## Constraints

### Primary Key

A primary key is a column or a set of columns that uniquely identifies each row in a table. A primary key must contain unique values and cannot contain NULL values. A table can have only one primary key, which can consist of one or more columns.

SQLite offers two ways to define a primary key: for one column or for multiple columns. The syntax for defining a primary key is as follows:

For only one column:

```sql
CREATE TABLE table_name (
    column1_name INTEGER NOT NULL PRIMARY KEY,
    ...
);
```

For multiple columns:

```sql
CREATE TABLE table_name (
    column1_name INTEGER NOT NULL,
    column2_name INTEGER NOT NULL,
    ...
    PRIMARY KEY (column1_name, column2_name)
);
```

Notice that for multiple columns, the primary key constraint is defined at the end of the table definition. This is because it applies to the combination of the two columns, not to each column individually.

When creating a table without specifying `WITHOUT ROWID`, SQLite automatically creates a hidden column called `rowid` that serves as the primary key for the table. This column is an integer that uniquely identifies each row in the table. If you define a primary key for the table of type `INTEGER`, SQLite will use that primary key instead of the `rowid` column as an alias.

Unlike other databases, SQLite does not allow to add a primary key constraint to an existing table using the `ALTER TABLE` command. Instead, you need to create a new table with the primary key constraint and copy the data from the old table to the new one. After that, you can drop the old table and rename the new one. To see more of how this works, check the material on this [SQLite tutorial website](https://www.sqlitetutorial.net/sqlite-primary-key/).



### NOT NULL

The `NOT NULL` constraint is used to ensure that a column cannot contain NULL values. This means that every row in the table must have a value for this column. If you try to insert a row without a value for a NOT NULL column, SQLite will raise an error.

```sql
CREATE TABLE table_name (
    column1_name INTEGER NOT NULL,
    column2_name TEXT NOT NULL,
    ...
);
```

### UNIQUE

The `UNIQUE` constraint is used to ensure that all values in a column are unique. This means that no two rows in the table can have the same value for this column. If you try to insert a row with a duplicate value for a UNIQUE column, SQLite will raise an error.

```sql
CREATE TABLE table_name (
    column1_name INTEGER UNIQUE,
    column2_name TEXT UNIQUE,
    ...
);
```

### DEFAULT

The `DEFAULT` constraint is used to specify a default value for a column. This means that if you do not provide a value for this column when inserting a row, SQLite will use the default value instead. The default value can be any valid expression, including a constant, a function call, or a subquery.

```sql
CREATE TABLE table_name (
    column1_name INTEGER DEFAULT 0,
    column2_name TEXT DEFAULT 'unknown',
    ...
);
```

### CHECK

The `CHECK` constraint is used to specify a condition that must be true for each row in the table. This means that if you try to insert a row that does not satisfy the condition, SQLite will raise an error. The condition can be any valid expression, including comparisons, logical operators, and functions.

```sql
CREATE TABLE table_name (
    column1_name INTEGER CHECK (column1_name > 0),
    column2_name TEXT CHECK (column2_name != ''),
    ...
);
```